In [1]:
import sys
sys.path.append('../pkg_mod')

import db_connector as db
import chemistry_psql as cp
import importlib
importlib.reload(cp)
import psycopg2
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdchem as rdch
from rdkit.Chem import Descriptors as rdd
from rdkit.Chem import rdMolDescriptors as rdmd
#from rdkit.Chem import SaltRemover as rdsr
from rdkit.Chem import PandasTools
import plotly.graph_objects as go

In [2]:
##### Global Variables #####

molprop = ['Atoms','MW', 'fSP3', 'HBA', 'HBD', 'Stereo Atoms', 'Rings', 'Rotatable Bonds', 'QED', 'logD','TPSA']

### Druglikeness Boundaries ###
### Lipinski
### Veber
### Ghosh
druglike_max = [55,500,0.5,10,5,1,3,10,0.6,5,140]
druglike_min = [30,160,0,0,0,0,0,6,0.4,0,0]

In [11]:
##### Function Definitions #####

### Calculate Descriptor Space

def collection_properties(dlist):

    d_temp = []
    line_count = 0
    
    if len(dlist) > 50:
        d_lines = min(int(len(dlist)/50),1000)
    else:
        d_lines = 50

    for i in dlist:
        smiles = i[0]
        m = i[1]
        numatoms = rdch.Mol.GetNumHeavyAtoms(m)
        mw = rdd.ExactMolWt(m)
        fsp3 = rdmd.CalcFractionCSP3(m)
        HBA = rdmd.CalcNumLipinskiHBA(m)
        HBD = rdmd.CalcNumLipinskiHBD(m)
        stereo = rdmd.CalcNumAtomStereoCenters(m)
        ring = rdmd.CalcNumRings(m)
        rotbond = rdmd.CalcNumRotatableBonds(m)
        qed = Chem.QED.qed(m)
        logd = Chem.Crippen.MolLogP(m)
        tpsa = rdmd.CalcTPSA(m)
        #print(smiles, numatoms,mw, fsp3, HBA, HBD, stereo, ring, rotbond, qed, logd)
        a = [smiles, numatoms,mw, fsp3, HBA, HBD, stereo, ring, rotbond, qed, logd,tpsa]
        d_temp.append(a)
        line_count = line_count+1
        if (line_count % d_lines) == 0:
            print('Lines processed : ', line_count)
    
    #df_drugs = pd.DataFrame(drug_temp, columns = ['smiles', 'numatoms','mw', 'fsp3', 'HBA', 'HBD', 'stereo', 'ring', 'rotbond', 'qed', 'logd','tpsa'])
    d = ['smiles']
    for i in molprop:
        d.append(i)
    
    #print(d)
    df_d = pd.DataFrame(d_temp, columns = d)
    
    return df_d

### Preparing the Compound Lists

def list_prep(dlist):
    temp_list = []
    line_count = 0
    
    if len(dlist) > 50:
        d_lines = min(int(len(dlist)/50),1000)
    else:
        d_lines = 50
        
    print(len(dlist), d_lines)

    for i in range(0, len(dlist)):
        j = dfr[i]
        #print(j, type(j))
        #print(line_count)
        if (line_count % d_lines) == 0:
            print('Lines processed : ', line_count)
        line_count = line_count+1
        if type(j) == str:
            j = j.split('.')
            for k in j:
                try:
                    #print(line_count, k)
                    m = Chem.MolFromSmiles(k)
                    numatoms = rdch.Mol.GetNumAtoms(m)
                    if numatoms >5:
                        #print(k, numatoms)
                        a = (k,m)
                        if a not in temp_list:
                            temp_list.append(a)
                except:
                    #print(j,k, ' - no valid SMILES/Molecule')
                    pass
        
        #line_count = line_count+1
            
    return temp_list
        

In [4]:
##### Read and Prepare Druglists

dfm = pd.read_csv("../Drug_Libraries/L1021_FDA_Drugs.csv", encoding = 'UTF-8')
dfr = dfm['SMILES']
#print(dfr)

drug_list = list_prep(dfr)
#print(drug_list)
print('Druglist Prepared')

df_drugs = collection_properties(drug_list)
print('Collection Properties')
print(df_drugs)

### Calculate Statistics

drugs_mean = df_drugs[molprop].mean()
drugs_std = df_drugs[molprop].std()
drugs_min = df_drugs[molprop].min()
drugs_max = df_drugs[molprop].max()
print(drugs_mean, type(drugs_mean))
print(drugs_std)
print(drugs_min)
print(drugs_max)

df_drugs.to_csv('../Drug_properties.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Drug_Libraries/L1021_FDA_Drugs.csv'

In [8]:
##### Read and Prepare DEL1 Structures

dfm = pd.read_csv("../DEL1_enum.csv")
dfr=dfm['Structure']

del1_list = list_prep(dfr)
#print(drug_list)
print('DEL1-list Prepared')

df_del1 = collection_properties(del1_list)
print('Collection Properties')
#print(df_del1)

### Calculate Statistics

del1_mean = df_del1[molprop].mean()
del1_std = df_del1[molprop].std()
del1_min = df_del1[molprop].min()
del1_max = df_del1[molprop].max()
print(del1_mean)
print(del1_std)
print(del1_min)
print(del1_max)

df_del1.to_csv('../DEL1_properties.csv')

995885 1000
Lines processed :  0
Lines processed :  1000
Lines processed :  2000
Lines processed :  3000
Lines processed :  4000
Lines processed :  5000
Lines processed :  6000
Lines processed :  7000
Lines processed :  8000
Lines processed :  9000
Lines processed :  10000
Lines processed :  11000
Lines processed :  12000
Lines processed :  13000
Lines processed :  14000
Lines processed :  15000
Lines processed :  16000
Lines processed :  17000
Lines processed :  18000
Lines processed :  19000
Lines processed :  20000
Lines processed :  21000
Lines processed :  22000
Lines processed :  23000
Lines processed :  24000
Lines processed :  25000
Lines processed :  26000
Lines processed :  27000
Lines processed :  28000
Lines processed :  29000
Lines processed :  30000
Lines processed :  31000
Lines processed :  32000
Lines processed :  33000
Lines processed :  34000
Lines processed :  35000
Lines processed :  36000
Lines processed :  37000
Lines processed :  38000
Lines processed :  39000
L

Lines processed :  319000
Lines processed :  320000
Lines processed :  321000
Lines processed :  322000
Lines processed :  323000
Lines processed :  324000
Lines processed :  325000
Lines processed :  326000
Lines processed :  327000
Lines processed :  328000
Lines processed :  329000
Lines processed :  330000
Lines processed :  331000
Lines processed :  332000
Lines processed :  333000
Lines processed :  334000
Lines processed :  335000
Lines processed :  336000
Lines processed :  337000
Lines processed :  338000
Lines processed :  339000
Lines processed :  340000
Lines processed :  341000
Lines processed :  342000
Lines processed :  343000
Lines processed :  344000
Lines processed :  345000
Lines processed :  346000
Lines processed :  347000
Lines processed :  348000
Lines processed :  349000
Lines processed :  350000
Lines processed :  351000
Lines processed :  352000
Lines processed :  353000
Lines processed :  354000
Lines processed :  355000
Lines processed :  356000
Lines proces

Lines processed :  635000
Lines processed :  636000
Lines processed :  637000
Lines processed :  638000
Lines processed :  639000
Lines processed :  640000
Lines processed :  641000
Lines processed :  642000
Lines processed :  643000
Lines processed :  644000
Lines processed :  645000
Lines processed :  646000
Lines processed :  647000
Lines processed :  648000
Lines processed :  649000
Lines processed :  650000
Lines processed :  651000
Lines processed :  652000
Lines processed :  653000
Lines processed :  654000
Lines processed :  655000
Lines processed :  656000
Lines processed :  657000
Lines processed :  658000
Lines processed :  659000
Lines processed :  660000
Lines processed :  661000
Lines processed :  662000
Lines processed :  663000
Lines processed :  664000
Lines processed :  665000
Lines processed :  666000
Lines processed :  667000
Lines processed :  668000
Lines processed :  669000
Lines processed :  670000
Lines processed :  671000
Lines processed :  672000
Lines proces

Lines processed :  951000
Lines processed :  952000
Lines processed :  953000
Lines processed :  954000
Lines processed :  955000
Lines processed :  956000
Lines processed :  957000
Lines processed :  958000
Lines processed :  959000
Lines processed :  960000
Lines processed :  961000
Lines processed :  962000
Lines processed :  963000
Lines processed :  964000
Lines processed :  965000
Lines processed :  966000
Lines processed :  967000
Lines processed :  968000
Lines processed :  969000
Lines processed :  970000
Lines processed :  971000
Lines processed :  972000
Lines processed :  973000
Lines processed :  974000
Lines processed :  975000
Lines processed :  976000
Lines processed :  977000
Lines processed :  978000
Lines processed :  979000
Lines processed :  980000
Lines processed :  981000
Lines processed :  982000
Lines processed :  983000
Lines processed :  984000
Lines processed :  985000
Lines processed :  986000
Lines processed :  987000
Lines processed :  988000
Lines proces

Lines processed :  275000
Lines processed :  276000
Lines processed :  277000
Lines processed :  278000
Lines processed :  279000
Lines processed :  280000
Lines processed :  281000
Lines processed :  282000
Lines processed :  283000
Lines processed :  284000
Lines processed :  285000
Lines processed :  286000
Lines processed :  287000
Lines processed :  288000
Lines processed :  289000
Lines processed :  290000
Lines processed :  291000
Lines processed :  292000
Lines processed :  293000
Lines processed :  294000
Lines processed :  295000
Lines processed :  296000
Lines processed :  297000
Lines processed :  298000
Lines processed :  299000
Lines processed :  300000
Lines processed :  301000
Lines processed :  302000
Lines processed :  303000
Lines processed :  304000
Lines processed :  305000
Lines processed :  306000
Lines processed :  307000
Lines processed :  308000
Lines processed :  309000
Lines processed :  310000
Lines processed :  311000
Lines processed :  312000
Lines proces

Lines processed :  591000
Lines processed :  592000
Lines processed :  593000
Lines processed :  594000
Lines processed :  595000
Lines processed :  596000
Lines processed :  597000
Lines processed :  598000
Lines processed :  599000
Lines processed :  600000
Lines processed :  601000
Lines processed :  602000
Lines processed :  603000
Lines processed :  604000
Lines processed :  605000
Lines processed :  606000
Lines processed :  607000
Lines processed :  608000
Lines processed :  609000
Lines processed :  610000
Lines processed :  611000
Lines processed :  612000
Lines processed :  613000
Lines processed :  614000
Lines processed :  615000
Lines processed :  616000
Lines processed :  617000
Lines processed :  618000
Lines processed :  619000
Lines processed :  620000
Lines processed :  621000
Lines processed :  622000
Lines processed :  623000
Lines processed :  624000
Lines processed :  625000
Lines processed :  626000
Lines processed :  627000
Lines processed :  628000
Lines proces

Lines processed :  907000
Lines processed :  908000
Lines processed :  909000
Lines processed :  910000
Lines processed :  911000
Lines processed :  912000
Lines processed :  913000
Lines processed :  914000
Lines processed :  915000
Lines processed :  916000
Lines processed :  917000
Lines processed :  918000
Lines processed :  919000
Lines processed :  920000
Lines processed :  921000
Lines processed :  922000
Lines processed :  923000
Lines processed :  924000
Lines processed :  925000
Lines processed :  926000
Lines processed :  927000
Lines processed :  928000
Lines processed :  929000
Lines processed :  930000
Lines processed :  931000
Lines processed :  932000
Lines processed :  933000
Lines processed :  934000
Lines processed :  935000
Lines processed :  936000
Lines processed :  937000
Lines processed :  938000
Lines processed :  939000
Lines processed :  940000
Lines processed :  941000
Lines processed :  942000
Lines processed :  943000
Lines processed :  944000
Lines proces

In [9]:
##### Read and Prepare DEL2 Structures

dfm = pd.read_csv("../DEL2_enum.csv")
dfr=dfm['Structure']

del2_list = list_prep(dfr)
#print(drug_list)
print('DEL2-list Prepared')

df_del2 = collection_properties(del2_list)
print('Collection Properties')
print(df_del2)

### Calculate Statistics

del2_mean = df_del2[molprop].mean()
del2_std = df_del2[molprop].std()
del2_min = df_del2[molprop].min()
del2_max = df_del2[molprop].max()
print(del2_mean)
print(del2_std)
print(del2_min)
print(del2_max)

df_del2.to_csv('../DEL2_properties.csv')

453379 1000
Lines processed :  0
Lines processed :  1000
Lines processed :  2000
Lines processed :  3000
Lines processed :  4000
Lines processed :  5000
Lines processed :  6000
Lines processed :  7000
Lines processed :  8000
Lines processed :  9000
Lines processed :  10000
Lines processed :  11000
Lines processed :  12000
Lines processed :  13000
Lines processed :  14000
Lines processed :  15000
Lines processed :  16000
Lines processed :  17000
Lines processed :  18000
Lines processed :  19000
Lines processed :  20000
Lines processed :  21000
Lines processed :  22000
Lines processed :  23000
Lines processed :  24000
Lines processed :  25000
Lines processed :  26000
Lines processed :  27000
Lines processed :  28000
Lines processed :  29000
Lines processed :  30000
Lines processed :  31000
Lines processed :  32000
Lines processed :  33000
Lines processed :  34000
Lines processed :  35000
Lines processed :  36000
Lines processed :  37000
Lines processed :  38000
Lines processed :  39000
L

Lines processed :  319000
Lines processed :  320000
Lines processed :  321000
Lines processed :  322000
Lines processed :  323000
Lines processed :  324000
Lines processed :  325000
Lines processed :  326000
Lines processed :  327000
Lines processed :  328000
Lines processed :  329000
Lines processed :  330000
Lines processed :  331000
Lines processed :  332000
Lines processed :  333000
Lines processed :  334000
Lines processed :  335000
Lines processed :  336000
Lines processed :  337000
Lines processed :  338000
Lines processed :  339000
Lines processed :  340000
Lines processed :  341000
Lines processed :  342000
Lines processed :  343000
Lines processed :  344000
Lines processed :  345000
Lines processed :  346000
Lines processed :  347000
Lines processed :  348000
Lines processed :  349000
Lines processed :  350000
Lines processed :  351000
Lines processed :  352000
Lines processed :  353000
Lines processed :  354000
Lines processed :  355000
Lines processed :  356000
Lines proces

Lines processed :  185000
Lines processed :  186000
Lines processed :  187000
Lines processed :  188000
Lines processed :  189000
Lines processed :  190000
Lines processed :  191000
Lines processed :  192000
Lines processed :  193000
Lines processed :  194000
Lines processed :  195000
Lines processed :  196000
Lines processed :  197000
Lines processed :  198000
Lines processed :  199000
Lines processed :  200000
Lines processed :  201000
Lines processed :  202000
Lines processed :  203000
Lines processed :  204000
Lines processed :  205000
Lines processed :  206000
Lines processed :  207000
Lines processed :  208000
Lines processed :  209000
Lines processed :  210000
Lines processed :  211000
Lines processed :  212000
Lines processed :  213000
Lines processed :  214000
Lines processed :  215000
Lines processed :  216000
Lines processed :  217000
Lines processed :  218000
Lines processed :  219000
Lines processed :  220000
Lines processed :  221000
Lines processed :  222000
Lines proces

In [14]:
##### Read and Prepare Zafrens Plate Structures

dfm = pd.read_csv("../ZafrensDrugs.csv")
dfr=dfm['smiles']

plate_list = list_prep(dfr)
#print(drug_list)
print('Zafrens Plate Prepared')

df_plate = collection_properties(plate_list)
print('Collection Properties')
print(df_plate)

### Calculate Statistics

plate_mean = df_plate[molprop].mean()
plate_std = df_plate[molprop].std()
plate_min = df_plate[molprop].min()
plate_max = df_plate[molprop].max()
print(plate_mean)
print(plate_std)
print(plate_min)
print(plate_max)

df_plate.to_csv('../Plate_properties.csv')

47 50
Lines processed :  0
Zafrens Plate Prepared
Collection Properties
                                               smiles  Atoms          MW  \
0   C1=CC=C(C=C1)C2=CC(=CC3=CC=C(O3)C4=CC=C(C=C4)[...     36  478.116486   
1   C1=CC=C(C(=C1)CC(=NNC2=NC(=CS2)C3=CC(=C(C=C3)C...     29  449.995631   
2                 C1=CC(=C(C=C1C(=O)O)[N+](=O)[O-])CO     14  197.032422   
3   CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...     24  349.109627   
4                             Nc1ccc(cc1)-c2ccc(N)cc2     14  184.100048   
5   CN(CC[C@@H](CC(=O)N[C@H]1C=C[C@@H](O[C@@H]1C(=...     30  422.202616   
6   COC1=CC=CC(=C1)C2=CC(=NC(=N2)N)NCC3=CC4=C(C=C3...     26  350.137890   
7   COc1cc2nccc(Oc3ccc(NC(=O)C4(C(=O)Nc5ccc(F)cc5)...     37  501.169999   
8   O=C(N[C@H](CO)[C@H](O)c1ccc([N+](=O)[O-])cc1)C...     20  322.012327   
9               CCCN(CCC)S(=O)(=O)C1=CC=C(C=C1)C(=O)O     19  285.103479   
10         CC1=CN(C(=O)NC1=O)C2CC(C(O2)CO)N=[N+]=[N-]     19  267.096754   
11  CC1=C(C=CC(=

In [ ]:
### Consolidated Writer when needed
df_drugs.to_csv('../Drug_properties.csv')
df_del1.to_csv('../DEL1_properties.csv')
df_del2.to_csv('../DEL2_properties.csv')
df_plate.to_csv('../Plate_properties.csv')

In [17]:
### Reader for Existing files and Statistics - bypass everything before
df_drugs = pd.read_csv('../Drug_properties.csv')
drugs_mean = df_drugs[molprop].mean()
drugs_std = df_drugs[molprop].std()
drugs_min = df_drugs[molprop].min()
drugs_max = df_drugs[molprop].max()
print("Drug Statistics")
print(drugs_mean)
print(drugs_std)
print(drugs_min)
print(drugs_max)
print()

df_del1 = pd.read_csv('../DEL1_properties.csv')
del1_mean = df_del1[molprop].mean()
del1_std = df_del1[molprop].std()
del1_min = df_del1[molprop].min()
del1_max = df_del1[molprop].max()
print("DEL1 Statistics")
print(del1_mean)
print(del1_std)
print(del1_min)
print(del1_max)
print()

df_del2 = pd.read_csv('../DEL2_properties.csv')
del2_mean = df_del2[molprop].mean()
del2_std = df_del2[molprop].std()
del2_min = df_del2[molprop].min()
del2_max = df_del2[molprop].max()
print("DEL2 Statistics")
print(del2_mean)
print(del2_std)
print(del2_min)
print(del2_max)
print()

df_plate = pd.read_csv('../Plate_properties.csv')
plate_mean = df_plate[molprop].mean()
plate_std = df_plate[molprop].std()
plate_min = df_plate[molprop].min()
plate_max = df_plate[molprop].max()
print("Zafrens Collection Statistics")
print(plate_mean)
print(plate_std)
print(plate_min)
print(plate_max)
print()

Drug Statistics
Atoms               23.233736
MW                 332.598974
fSP3                 0.413248
HBA                  5.814591
HBD                  2.435874
Stereo Atoms         1.779275
Rings                2.495353
Rotatable Bonds      4.810409
QED                  0.543880
logD                 1.995642
TPSA                85.862054
dtype: float64
Atoms               12.390542
MW                 176.153191
fSP3                 0.278008
HBA                  3.991915
HBD                  2.685009
Stereo Atoms         3.500027
Rings                1.677642
Rotatable Bonds      4.118460
QED                  0.207273
logD                 2.609771
TPSA                64.321491
dtype: float64
Atoms               6.000000
MW                 82.053098
fSP3                0.000000
HBA                 0.000000
HBD                 0.000000
Stereo Atoms        0.000000
Rings               0.000000
Rotatable Bonds     0.000000
QED                 0.011297
logD               -8.895300
TPSA

In [19]:
ax = molprop.copy()
scale = [1,1,1,1,1,1,1,1,1,1,1]

##### Calculate Axes and Scales for Plots #####

axmin = []
axmax = []

for i in range(0,len(molprop)):
    #print(i)
    if ax[i] in ['qed','fsp3']:
        axmin.append(0)
        axmax.append(1)
    else:
        amin = min(druglike_max[i], drugs_min[i], del2_min[i], del1_min[i], plate_min[i])
        if amin > 0:
            amin = 0
        amax = max(druglike_max[i], drugs_max[i], del2_max[i], del1_max[i],plate_max[i])
        scale[i] = amax - amin
        axmin.append(amin)
        axmax.append(amax)
                   
print('Scales : ',scale)
print()
print('Maxima : ',axmax)
print()
print('Minima : ',axmin)


Scales :  [115.0, 1663.492351879999, 1.0, 43.0, 25.0, 30.0, 13.0, 35.0, 0.9437114379351168, 26.74919999999999, 702.02]

Maxima :  [115.0, 1663.492351879999, 1.0, 43.0, 25.0, 30.0, 13.0, 35.0, 0.9437114379351168, 17.853899999999975, 702.02]

Minima :  [0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, -8.895300000000017, 0.0]


In [21]:
##### Calculate trace data #####

std_fact = 2

### Druglikeness

dl_min = []
dl_max = []

for i in range(0,len(molprop)):
    dmax = (druglike_max[i]-axmin[i])/scale[i]
    dmin = (druglike_min[i]-axmin[i])/scale[i]
    dl_max.append(dmax)
    dl_min.append(dmin)

print("Druglikeness")
print(dl_min)
print(dl_max)
print()

### Drug Collection

drug_min = []
drug_max = []

for i in range(0,len(molprop)):
    dmax = drugs_mean[i] + std_fact*drugs_std[i]
    dmin = drugs_mean[i] - std_fact*drugs_std[i]
    #print(molprop[i],drugs_mean[i],drugs_std[i],dmax,dmin)
    dmax = (dmax-axmin[i])/scale[i]
    dmin = (dmin-axmin[i])/scale[i]
    if dmin<0:
        dmin = 0
    #print(molprop[i],drugs_mean[i],drugs_std[i],dmax,dmin)
    drug_max.append(dmax)
    drug_min.append(dmin)
    
print("Drugs")
print(drug_min)
print(drug_max)
print()

### DEL2 Collection

del2p_min = []
del2p_max = []

for i in range(0,len(molprop)):
    dmax = del2_mean[i] + std_fact*del2_std[i]
    dmin = del2_mean[i] - std_fact*del2_std[i]    
    dmax = (dmax-axmin[i])/scale[i]
    dmin = (dmin-axmin[i])/scale[i]
    if dmin<0:
        dmin = 0
    del2p_max.append(dmax)
    del2p_min.append(dmin)
    
print("DEL2")
print(del2p_min)
print(del2p_max)
print()

### DEL1 Collection

del1p_min = []
del1p_max = []

for i in range(0,len(molprop)):
    dmax = del1_mean[i] + std_fact*del1_std[i]
    dmin = del1_mean[i] - std_fact*del1_std[i]    
    dmax = (dmax-axmin[i])/scale[i]
    dmin = (dmin-axmin[i])/scale[i]
    if dmin<0:
        dmin = 0
    del1p_max.append(dmax)
    del1p_min.append(dmin)

print("DEL1")
print(del1p_min)
print(del1p_max)
print()

### Zafrens Plates

platep_min = []
platep_max = []

for i in range(0,len(molprop)):
    dmax = plate_mean[i] + std_fact*plate_std[i]
    dmin = plate_mean[i] - std_fact*plate_std[i]    
    dmax = (dmax-axmin[i])/scale[i]
    dmin = (dmin-axmin[i])/scale[i]
    if dmin<0:
        dmin = 0
    platep_max.append(dmax)
    platep_min.append(dmin)
    
print("Zafrens")
print(platep_min)
print(platep_max)

Druglikeness
[0.2608695652173913, 0.09618318943226623, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17142857142857143, 0.4238583786535618, 0.33254452469606643, 0.0]
[0.4782608695652174, 0.30057246697583195, 0.5, 0.23255813953488372, 0.2, 0.03333333333333333, 0.23076923076923078, 0.2857142857142857, 0.6357875679803426, 0.5194660027215775, 0.19942451782000514]

Drugs
[0, 0, 0, 0, 0, 0, 0, 0, 0.13704849898035934, 0.21202128684158728, 0]
[0.41752017882356696, 0.41172738538861786, 0.9692648073649767, 0.3208935348809258, 0.3122356893415111, 0.2926442992395945, 0.45004903972139626, 0.3727808349471888, 1.0155910461149467, 0.6022791095653641, 0.3055540241754811]

DEL2
[0.16656445266628966, 0.16698522876846242, 0.15335544598544854, 0.1260628662197918, 0.10850540151015825, 0.03305787196237301, 0.0010220197217084986, 0.1194904857024104, 0.04005039606087427, 0.21667477802003848, 0.12468700891588404]
[0.38962655178781114, 0.3754008126954943, 0.8046372501099088, 0.3446793975127043, 0.35623139648180424, 0.1683405196666

In [23]:
##### Plot Radar Plots #####
ax_max = [1,1,1,1,1,1,1,1,1,1,1,1]
### Close Traces
ax.append(ax[0])
dl_max.append(dl_max[0])
dl_min.append(dl_min[0])
drug_max.append(drug_max[0])
drug_min.append(drug_min[0])
del2p_max.append(del2p_max[0])
del2p_min.append(del2p_min[0])
del1p_max.append(del1p_max[0])
del1p_min.append(del1p_min[0])
platep_max.append(platep_max[0])
platep_min.append(platep_min[0])

#print(drug_max)
#print(drug_min)

### Drugs & Druglikeness ###

fig_drug_like = go.Figure()

fig_drug_like.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_drug_like.add_trace(go.Scatterpolar(name = 'Drug Collection', showlegend=False,
  r=drug_min,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_drug_like.add_trace(go.Scatterpolar(name = 'Drug Collection',
  r=drug_max,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_drug_like.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)', showlegend=False,
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    marker = dict(size = 1),
    line = dict(color = 'red')
  #fill='none'
))

fig_drug_like.add_trace(go.Scatterpolar(name = 'Druglikeness',
  r=dl_max,
  theta=ax,
    line = dict(color = 'red'),
    opacity = .5,
    marker = dict(size = 1),
    fillcolor = 'red',
  fill='tonext'
))




fig_drug_like.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_drug_like.show()

### DEL2 & Druglikeness ###

fig_del2_like = go.Figure()

fig_del2_like.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))

fig_del2_like.add_trace(go.Scatterpolar(name = 'DEL2 Library', showlegend=False,
  r=del2p_min,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del2_like.add_trace(go.Scatterpolar(name = 'Zafrens Library 2',
  r=del2p_max,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_del2_like.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)', showlegend=False,
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    marker = dict(size = 1),
    line = dict(color = 'red')
  #fill='none'
))

fig_del2_like.add_trace(go.Scatterpolar(name = 'Druglikeness',
  r=dl_max,
  theta=ax,
    line = dict(color = 'red'),
    opacity = .5,
    marker = dict(size = 1),
    fillcolor = 'red',
  fill='tonext'
))




fig_del2_like.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_del2_like.show()

### DEL2 & Drug ###

fig_del2_drug = go.Figure()

fig_del2_drug.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))

fig_del2_drug.add_trace(go.Scatterpolar(name = 'Drug Collection', showlegend=False,
  r=drug_min,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del2_drug.add_trace(go.Scatterpolar(name = 'Drug Collection',
  r=drug_max,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_del2_drug.add_trace(go.Scatterpolar(name = 'DEL2 Library', showlegend=False,
  r=del2p_min,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del2_drug.add_trace(go.Scatterpolar(name = 'Zafrens Library 2',
  r=del2p_max,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))


fig_del2_drug.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_del2_drug.show()


### DEL2 & Drug & Druglikeness ###

fig_del2_drug_like = go.Figure()

fig_del2_drug.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))

fig_del2_drug_like.add_trace(go.Scatterpolar(name = 'Drug Collection', showlegend=False,
  r=drug_min,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del2_drug_like.add_trace(go.Scatterpolar(name = 'Drug Collection',
  r=drug_max,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_del2_drug_like.add_trace(go.Scatterpolar(name = 'DEL2 Library', showlegend=False,
  r=del2p_min,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del2_drug_like.add_trace(go.Scatterpolar(name = 'Zafrens Library 2',
  r=del2p_max,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_del2_drug_like.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)', showlegend=False,
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    marker = dict(size = 1),
    line = dict(color = 'red')
  #fill='none'
))

fig_del2_drug_like.add_trace(go.Scatterpolar(name = 'Druglikeness',
  r=dl_max,
  theta=ax,
    line = dict(color = 'red'),
    opacity = .5,
    marker = dict(size = 1),
    fillcolor = 'red',
  fill='tonext'
))


fig_del2_drug_like.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_del2_drug_like.show()


### DEL1 & Druglikeness ###

fig_del1_like = go.Figure()

fig_del1_like.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))

fig_del1_like.add_trace(go.Scatterpolar(name = 'DEL2 Library', showlegend=False,
  r=del1p_min,
  theta=ax,
    line = dict(width = 2, color = 'yellow'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del1_like.add_trace(go.Scatterpolar(name = 'Zafrens Library 1',
  r=del1p_max,
  theta=ax,
    line = dict(width = 2, color = 'yellow'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_del1_like.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)', showlegend=False,
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    marker = dict(size = 1),
    line = dict(color = 'red')
  #fill='none'
))

fig_del1_like.add_trace(go.Scatterpolar(name = 'Druglikeness',
  r=dl_max,
  theta=ax,
    line = dict(color = 'red'),
    opacity = .5,
    marker = dict(size = 1),
    fillcolor = 'red',
  fill='tonext'
))


fig_del1_like.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_del1_like.show()

### DEL1 & Drug ###

fig_del1_drug = go.Figure()

fig_del1_drug.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))

fig_del1_drug.add_trace(go.Scatterpolar(name = 'Drug Collection', showlegend=False,
  r=drug_min,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del1_drug.add_trace(go.Scatterpolar(name = 'Drug Collection',
  r=drug_max,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_del1_drug.add_trace(go.Scatterpolar(name = 'DEL1 Library', showlegend=False,
  r=del1p_min,
  theta=ax,
    line = dict(width = 2, color = 'yellow'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del1_drug.add_trace(go.Scatterpolar(name = 'Zafrens Library 1',
  r=del1p_max,
  theta=ax,
    line = dict(width = 2, color = 'yellow'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))


fig_del1_drug.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_del1_drug.show()

### DEL1 & DEL2 ###

fig_del1_del2 = go.Figure()

fig_del1_del2.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))

fig_del1_del2.add_trace(go.Scatterpolar(name = 'DEL2 Library', showlegend=False,
  r=del2p_min,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del1_del2.add_trace(go.Scatterpolar(name = 'Zafrens Library 2',
  r=del2p_max,
  theta=ax,
    line = dict(width = 2, color = 'green'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))



fig_del1_del2.add_trace(go.Scatterpolar(name = 'DEL1 Library', showlegend=False,
  r=del1p_min,
  theta=ax,
    line = dict(width = 2, color = 'yellow'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_del1_del2.add_trace(go.Scatterpolar(name = 'Zafrens Library 1',
  r=del1p_max,
  theta=ax,
    line = dict(width = 2, color = 'yellow'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))


fig_del1_del2.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_del1_del2.show()



### Zafrems Coll & Druglikeness ###

fig_plate_like = go.Figure()

fig_plate_like.add_trace(go.Scatterpolar(name = 'leveling', showlegend=False, #visible = False,
  r=ax_max,
  theta=ax,
    line = dict(width = 0, color = 'silver'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_plate_like.add_trace(go.Scatterpolar(name = 'Zafrens Collection', showlegend=False,
  r=platep_min,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig_plate_like.add_trace(go.Scatterpolar(name = 'Zafrens Collection',
  r=platep_max,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
    #opacity = .25,
  fill='tonext'#, fillcolor ='lightblue'
))

fig_plate_like.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)', showlegend=False,
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    marker = dict(size = 1),
    line = dict(color = 'red')
  #fill='none'
))

fig_plate_like.add_trace(go.Scatterpolar(name = 'Druglikeness',
  r=dl_max,
  theta=ax,
    line = dict(color = 'red'),
    opacity = .5,
    marker = dict(size = 1),
    fillcolor = 'red',
  fill='tonext'
))

fig_plate_like.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig_plate_like.show()


In [ ]:
val = []
ax = molprop.copy()
scale = [1,1,1,1,1,1,1,1,1,1,1]
val_width = [5,12,15,8,16,32,64,10,20,40,10]
maxval = 0

dl_max = druglike_max
dl_min = druglike_min

for i in range(0,len(drugs_mean)):
        amin = drugs_min[i]
        if amin > 0:
            amin = 0
        amax = drugs_max[i]
        scale[i] = amax - amin
        j = (j-amin)/scale[i]
        val.append(j)
        dl_max[i] = (druglike_max[i]-amin)/scale[i]
        dl_min[i] = (druglike_min[i]-amin)/scale[i]
        val_width[i] = (100*drugs_std[i])/scale[i]
        maxval = max(j, maxval)

print(scale)
print(dl_max)
print(dl_min)  
print(val)
print(val_width)


ax.append(ax[0])
val.append(val[0])
dl_max.append(dl_max[0])
dl_min.append(dl_min[0])

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)',
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    line = dict(color = 'green')
  #fill='none'
))

fig.add_trace(go.Scatterpolar(name = 'Druglikeness (upper)',
  r=dl_max,
  theta=ax,
    line = dict(color = 'green'),
    opacity = .5,
    fillcolor = 'lightgreen',
  fill='tonext'
))


fig.add_trace(go.Scatterpolar(name = 'Drug Collection',
  r=val,
  theta=ax,
    line = dict(width = 5, color = 'blue'),
    marker = dict(size=val_width, sizeref = maxval, sizemode = 'diameter', color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    
  #fill='toself'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig.show()

In [ ]:
val_max = []
val_min = []

for i in range(0,len(val)-1):
    j = val[i]+drugs_std[i]/scale[i]
    val_max.append(j)
    j = val[i]-drugs_std[i]/scale[i]
    val_min.append(j)
    
val_min.append(val_min[0])
val_max.append(val_max[0])
    
print(val_min)
print(val_max)
    
fig2 = go.Figure()

fig2.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)', showlegend=False,
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    line = dict(color = 'green')
  #fill='none'
))

fig2.add_trace(go.Scatterpolar(name = 'Druglikeness (upper)',
  r=dl_max,
  theta=ax,
    line = dict(color = 'green'),
    opacity = .5,
    fillcolor = 'lightgreen',
  fill='tonext'
))


fig2.add_trace(go.Scatterpolar(name = 'Drug Collection', showlegend=False,
  r=val_min,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig2.add_trace(go.Scatterpolar(name = 'Drug Collection',
  r=val_max,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
  fill='tonext'
))

fig2.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig2.show()

In [ ]:
fig.show()
fig2.show()

In [ ]:
print(dir(rdmd.Mol))

In [ ]:
dfm = pd.read_csv("../DEL2_enum.csv")
dfr=dfm['Structure']
print(dfr)

del2_list = []

for i in range(0, len(dfr)):
    j = dfr[i]
    #print(j, type(j))
    if type(j) == str:
        j = j.split('.')
        for k in j:
            try:
                #print(k)
                m = Chem.MolFromSmiles(k)
                numatoms = rdch.Mol.GetNumAtoms(m)
                if numatoms >5:
                    #print(k, numatoms)
                    a = (k,m)
                    if a not in drug_list:
                        del2_list.append(a)
            except:
                #print(j,k, ' - no valid SMILES/Molecule')
                pass
        
print()
print(len(del2_list))
#print()
#print(del2_list)

In [ ]:
drug_temp = []


for i in del2_list:
    smiles = i[0]
    m = i[1]
    numatoms = rdch.Mol.GetNumHeavyAtoms(m)
    mw = rdd.ExactMolWt(m)
    fsp3 = rdmd.CalcFractionCSP3(m)
    HBA = rdmd.CalcNumLipinskiHBA(m)
    HBD = rdmd.CalcNumLipinskiHBD(m)
    stereo = rdmd.CalcNumAtomStereoCenters(m)
    ring = rdmd.CalcNumRings(m)
    rotbond = rdmd.CalcNumRotatableBonds(m)
    qed = Chem.QED.qed(m)
    logd = Chem.Crippen.MolLogP(m)
    tpsa = rdmd.CalcTPSA(m)
    #print(smiles, numatoms,mw, fsp3, HBA, HBD, stereo, ring, rotbond, qed, logd)
    a = [smiles, numatoms,mw, fsp3, HBA, HBD, stereo, ring, rotbond, qed, logd, tpsa]
    drug_temp.append(a)
    
df_del2 = pd.DataFrame(drug_temp, columns = ['smiles', 'numatoms','mw', 'fsp3', 'HBA', 'HBD', 'stereo', 'ring', 'rotbond', 'qed', 'logd', 'tpsa'])

print(df_del2)

In [ ]:
del2_mean = df_del2[['numatoms','mw', 'fsp3', 'HBA', 'HBD', 'stereo', 'ring', 'rotbond', 'qed', 'logd']].mean()
del2_std = df_del2[['numatoms','mw', 'fsp3', 'HBA', 'HBD', 'stereo', 'ring', 'rotbond', 'qed', 'logd']].std()
del2_min = df_del2[['numatoms','mw', 'fsp3', 'HBA', 'HBD', 'stereo', 'ring', 'rotbond', 'qed', 'logd']].min()
del2_max = df_del2[['numatoms','mw', 'fsp3', 'HBA', 'HBD', 'stereo', 'ring', 'rotbond', 'qed', 'logd']].max()
print(del2_mean)
print(del2_std)
print(del2_min)
print(drugs_max)

val = []
ax = molprop.copy()
#dl_max = [55,500,0.5,10,5,1,3,10,0.6,5]
#dl_min = [30,160,0,0,0,0,0,6,0.4,0]
scale = [1,1,1,1,1,1,1,1,1,1]
val_width = [5,12,15,8,16,32,64,10,20,40]
val_min = []
val_max = []

for i in range(0,len(del2_mean)):
    val_width[i] = del2_std[i]

for i in range(0,len(del2_mean)):
    #print(i)
    j = del2_mean[i]
    if ax[i] in ['qed','fsp3']:
        val.append(j)
        jmin = j-val_width[i]
        jmax = j+val_width[i]
        val_min.append(jmin)
        val_max.append(jmax)
        print(i,j, ax[i])
        val_width[i] = del2_std[i]
    else:
        amin = min(drugs_min[i],druglike_min[i],del2_min[i])
        if amin > 0:
            amin = 0
        amax = max(drugs_max[i],druglike_max[i],del2_max[i])
        scale[i] = amax - amin
        jmin = (j-val_width[i]-amin)/scale[i]
        jmax = (j+val_width[i]-amin)/scale[i]
        val_min.append(jmin)
        val_max.append(jmax)
        j = (j-amin)/scale[i]
        val.append(j)
        dl_max[i] = (druglike_max[i]-amin)/scale[i]
        dl_min[i] = (druglike_min[i]-amin)/scale[i]
        val_width[i] = del2_std[i]/scale[i]

print()
print(scale)
print(val)
print('Likeness')
print(dl_max)
print(dl_min)  
print('Drugs')
print(val_min)
print(val_max)

ax.append(ax[0])
val.append(val[0])
val_min.append(val_min[0])
val_max.append(val_max[0])
dl_max.append(dl_max[0])
dl_min.append(dl_min[0])

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)',
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    line = dict(color = 'green')
  #fill='none'
))

fig.add_trace(go.Scatterpolar(name = 'Druglikeness (upper)',
  r=dl_max,
  theta=ax,
    line = dict(color = 'green'),
    opacity = .5,
    fillcolor = 'silver',
  fill='tonext'
))


fig.add_trace(go.Scatterpolar(name = 'DEL2',
  r=val,
  theta=ax,
    line = dict(width = 5, color = 'blue'),
    marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    
  #fill='toself'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig.show()


##### Double Trace #####

#val_max = []

#for i in val:
#    j = i+.1
#    val_max.append(j)
    
fig2 = go.Figure()

fig2.add_trace(go.Scatterpolar(
    name = 'Druglikeness (lower)', showlegend=False,
  r=dl_min,
  theta=ax,
    opacity =.25,
    #fillcolor = 'white',
    line = dict(color = 'green')
  #fill='none'
))

fig2.add_trace(go.Scatterpolar(name = 'Druglikeness (upper)',
  r=dl_max,
  theta=ax,
    line = dict(color = 'green'),
    opacity = .5,
    fillcolor = 'silver',
  fill='tonext'
))


fig2.add_trace(go.Scatterpolar(name = 'Del2', showlegend=False,
  r=val_min,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1)
    
  #fill='toself'
))


fig2.add_trace(go.Scatterpolar(name = 'DEL2',
  r=val_max,
  theta=ax,
    line = dict(width = 2, color = 'blue'),
    #marker = dict(size=val_width, color = 'blue', symbol = 'circle', gradient = dict(type = 'radial',color = 'lightblue'))
    marker = dict(size = 1),
  fill='tonext'
))

fig2.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig2.show()

In [ ]:
import sys
sys.path

In [ ]:
sys.path.append('../pkg_mod')

In [ ]:
struct_list = []
print(struct_raw)
print()

for i in struct_raw:
    #print(i)
    a = i[0].split('.')
    #print(a)
    for j in a:
        b = (j,)
        #print(j, b)
        if b not in struct_list:
            struct_list.append(b)

print(struct_list)
        

In [ ]:
dfm = pd.read_excel("../Drug_Libraries/L1021-DiscoveryProbe-FDA-approved-Drug-Library.xlsx", sheet_name=1)
#print(dfm)
dfr = dfm['SMILES']
#print(dfr)

drug_list = list_prep(dfr)

for i in range(0, len(dfr)):
    j = dfr[i]
    #print(j, type(j))
    if type(j) == str:
        j = j.split('.')
        for k in j:
            try:
                #print(k)
                m = Chem.MolFromSmiles(k)
                numatoms = rdch.Mol.GetNumAtoms(m)
                if numatoms >5:
                    #print(k, numatoms)
                    a = (k,m)
                    if a not in drug_list:
                        drug_list.append(a)
            except:
                #print(j,k, ' - no valid SMILES/Molecule')
                pass
        

print(len(drug_list))
print()
print(drug_list)

In [ ]:
drug_temp = []


for i in drug_list:
    smiles = i[0]
    m = i[1]
    numatoms = rdch.Mol.GetNumHeavyAtoms(m)
    mw = rdd.ExactMolWt(m)
    fsp3 = rdmd.CalcFractionCSP3(m)
    HBA = rdmd.CalcNumLipinskiHBA(m)
    HBD = rdmd.CalcNumLipinskiHBD(m)
    stereo = rdmd.CalcNumAtomStereoCenters(m)
    ring = rdmd.CalcNumRings(m)
    rotbond = rdmd.CalcNumRotatableBonds(m)
    qed = Chem.QED.qed(m)
    logd = Chem.Crippen.MolLogP(m)
    tpsa = rdmd.CalcTPSA(m)
    #print(smiles, numatoms,mw, fsp3, HBA, HBD, stereo, ring, rotbond, qed, logd)
    a = [smiles, numatoms,mw, fsp3, HBA, HBD, stereo, ring, rotbond, qed, logd,tpsa]
    drug_temp.append(a)
    
#df_drugs = pd.DataFrame(drug_temp, columns = ['smiles', 'numatoms','mw', 'fsp3', 'HBA', 'HBD', 'stereo', 'ring', 'rotbond', 'qed', 'logd','tpsa'])
d = ['smiles']
for i in molprop:
    d.append(i)
    
print(d)
    
df_drugs = pd.DataFrame(drug_temp, columns = d)
print(df_drugs)